# iStock Crawler

In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import random
import time
import re
import os
import base64

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
import wget

## Import breeds from txt file

In [ ]:
final_breeds = []

path = os.getcwd()
my_file = open(os.path.join(path ,"Breed_iStock.txt"), "r")
content = my_file.read()
content_list = content.split("\n")
my_file.close()

for breed in content_list:
        word = breed.replace("_", " ").replace("-", " ")
        word= word[:1].upper() + word[1:]
        final_breeds.append(word)
        
print(final_breeds)

## Selenium driver setup

In [ ]:
chrome_options = webdriver.ChromeOptions()
s = Service('C:/Users/Asus/Documents/WebDriver/chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get('https://www.istockphoto.com/')
time.sleep(5)

In [ ]:
search_URL = "https://www.istockphoto.com/search/2/image?phrase="
main_URL = "https://www.istockphoto.com/"
page_url = "&page="

## iStock Crawling/Scraping  Functions

In [ ]:
def create_folder(query):
    if not os.path.exists(os.getcwd() + '/' + query):
        path = os.getcwd()
        path = os.path.join(path, query)
        os.mkdir(path)
        print(query + " Folder Created")
    else:
        print(query + " Folder Already Exist")
        
def save_images(images, query):
    counter = 0
    path = os.getcwd()
    path = os.path.join(path, query)
    for img in images:
        save_as = os.path.join(path ,query + str(counter) +'.jpg')
        try:
            wget.download(img, save_as)
        except Exception as e:
            print("Could not download file %s"%(img))
            print(e)
            continue
        counter += 1
    
    print("%s Images Saved! | Total of: %s Images" %(query,counter))
    
def save_single_images(img, query):
    counter = 0
    path = os.getcwd()
    path = os.path.join(path, query)
    save_as = os.path.join(path ,query + str(counter) +'.jpg')
    print(img)
    wget.download(img, save_as)
    
    print("%s Images Saved!" %(query))

## Use The Automation For Each Breed

#### In this automation we used Selenium and BeautifulSoup combined

In [ ]:
wait = WebDriverWait(driver, 10)

for breed in final_breeds:
    
    print("Search images for: " + breed)
    img_urls = []
    img_download_links = []
    query = breed
    page = driver.get(search_URL + query + page_url + "1") 

    time.sleep(random.randint(3,6))
    page_source = driver.page_source
    soup = bs(page_source, 'html.parser')
    total_pages = soup.find('span',{'class':'PaginationRow-module__lastPage___FVgO3'}).text
    print("Total pages to run: %s"%(total_pages))
    
    for i in range(1, int(total_pages)+1):
        page_source = driver.page_source
        soup = bs(page_source, 'html.parser')
        links = soup.find_all('img',{'class':'MosaicAsset-module__thumb___tdc6z'})
        for link in links:
            img_download_links.append(link['src'])
        print("Images: %s"%(len(img_download_links)))
        if(len(img_download_links) > 12000):
            break
        time.sleep(random.randint(2,4))
        page = driver.get(search_URL + query + page_url + str(i+1))
    
    folder_name = query + "iStock"
    #create path if dosent exsist
    create_folder(folder_name)
    
    #save the images
    save_images(img_download_links,folder_name)
    print("Donwload for %s finished, Total of %s images!"%(breed,len(img_download_links)))
    #save_single_images(img_download_links[0], query)

### In this project all the data came from scrapping and the crawling is used for educational purpose only.